In [1]:
# !open .

In [2]:
import pandas as pd
import requests
import bs4
import math
import time
import matplotlib.pyplot as plt
from matplotlib import font_manager, rc
pd.set_option('display.max_colwidth', -1)
rc('font', family='AppleGothic')
plt.rcParams['axes.unicode_minus'] = False
import NeoQuant as nq

## get all company list

In [3]:
companies = nq.get_all_company_list()
companies.index = 'A' + companies.index

In [4]:
sector_df = pd.read_excel(r'data/s_rim_sector.xlsx')
sector_df.index = sector_df[sector_df.columns[0]]
sector_df.index.name = ''
sector_df.drop(sector_df.columns[0], axis = 1, inplace = True)

## 수익률 계산

In [5]:
port_df = sector_df

In [6]:
import time

port_list = port_df.index
total = len(port_list)
for num, company_code in enumerate(port_list):
    company_name = nq.get_company_name(company_code, companies)
    print(str(num + 1) + ' / ' + str(total) + " : " + company_name + ' (' + str(company_code) + ')')
    time.sleep(1)
    try:
        price_df = nq.request_price_list(company_code, 'day', 100)
    except requests.exceptions.Timeout:
        time.sleep(60)
        price_df = nq.request_price_list(company_code, 'day', 100)
        
    current = price_df[price_df.columns[0]][-1]
    port_df.at[company_code, 'current'] = int(current)

1 / 63 : DL (A000210)
2 / 63 : KG ETS (A151860)
3 / 63 : HDC (A012630)
4 / 63 : JB금융지주 (A175330)
5 / 63 : 대원산업 (A005710)
6 / 63 : 디티알오토모티브 (A007340)
7 / 63 : 한신공영 (A004960)
8 / 63 : 두산 (A000150)
9 / 63 : 상상인 (A038540)
10 / 63 : 리드코프 (A012700)
11 / 63 : 코텍 (A052330)
12 / 63 : 메리츠금융지주 (A138040)
13 / 63 : KG케미칼 (A001390)
14 / 63 : 계룡건설산업 (A013580)
15 / 63 : 삼양통상 (A002170)
16 / 63 : AJ네트웍스 (A095570)
17 / 63 : 한국토지신탁 (A034830)
18 / 63 : 신대양제지 (A016590)
19 / 63 : 삼호개발 (A010960)
20 / 63 : 토비스 (A051360)
21 / 63 : 디씨엠 (A024090)
22 / 63 : 동원개발 (A013120)
23 / 63 : 한미글로벌 (A053690)
24 / 63 : 메리츠증권 (A008560)
25 / 63 : 대상홀딩스 (A084690)
26 / 63 : 창해에탄올 (A004650)
27 / 63 : HDC현대산업개발 (A294870)
28 / 63 : 한국자산신탁 (A123890)
29 / 63 : 금화피에스시 (A036190)
30 / 63 : 메리츠화재 (A000060)
31 / 63 : 우리금융캐피탈 (A033660)
32 / 63 : 삼영무역 (A002810)
33 / 63 : 아세아제지 (A002310)
34 / 63 : 동양이엔피 (A079960)
35 / 63 : 서희건설 (A035890)
36 / 63 : 다우기술 (A023590)
37 / 63 : 케이씨 (A029460)
38 / 63 : 다우데이타 (A032190)
39 / 63 : CNH (A023460)
40 / 63

In [7]:
profit_rate = (port_df['current'] - port_df['price']) / port_df['price']
profit_rate = profit_rate.dropna()
profit_rate = profit_rate.map(lambda rate: str("%.2f" % round(float(rate * 100), 2)) + '%')
port_df['수익률'] = profit_rate

In [8]:
profit_rate = port_df['수익률'].map(lambda rate: float(rate[:-1]))
average_profit = profit_rate.sum() / len(port_df)
print('average profit : ' + "%.2f" % average_profit + '%')

average profit : 0.81%


In [16]:
port_df['profite_rate'] = profit_rate
port_df.sort_values(by='profite_rate', ascending=False)

In [ ]:
port_df.to_excel(r'data/s_rim_sector_profit.xlsx')

## show chart

In [ ]:
company_code_list = port_df.index

In [ ]:
for company_code in company_code_list:
    company_name = nq.get_company_name(company_code, companies)

    price_df = nq.request_price_list(company_code, 'day', 1000)

    price_high = sector_df.loc[company_code]['high']
    price_middle = sector_df.loc[company_code]['middle']
    price_low = sector_df.loc[company_code]['low']
    price_very_low = sector_df.loc[company_code]['very_low']

    price_df['price_very_low'] = [price_very_low] * len(price_df)
    price_df['price_low'] = [price_low] * len(price_df)
    price_df['price_middle'] = [price_middle] * len(price_df)
    price_df['price_high'] = [price_high] * len(price_df)

    nq.show_price_chart(company_code, company_name, price_df)